# Background

## DNA Melting Point

> Colab version: https://colab.research.google.com/drive/1YBbVp0rSQiY76YzYzlxF1gOQXBi1sc1U#scrollTo=PsDHUoNGU62n
> 
> ----
> Original Version: https://github.com/ramanathanlab/SciFM24-Tutorial/blob/main/notebooks/GenSLM_Downstream.ipynb
> Original Colab: https://colab.research.google.com/github/ramanathanlab/SciFM24-Tutorial/blob/main/notebooks/GenSLM_Downstream.ipynb

__Wikipedia:__

Nucleic acid thermodynamics is the study of how temperature affects the nucleic acid structure of double-stranded DNA (dsDNA). The melting temperature (Tm) is defined as the temperature at which half of the DNA strands are in the random coil or single-stranded (ssDNA) state. Tm depends on the length of the DNA molecule and its specific nucleotide sequence. DNA, when in a state where its two strands are dissociated (i.e., the dsDNA molecule exists as two independent strands), is referred to as having been denatured by the high temperature.


## This notebook

In this notebook we will use the GenSLM 25M parameter langauge model to generate embeddings for sequences and use a downstream model to take the embeddings and predict the melting point of the associated sequence. This workflow is common for many bioinformatics tasks, and can easily be adapted to other regression and classification problems.

In [17]:
!python --version

Python 3.10.14


In [18]:
# Installing GenSLM
# NOTE: You may need to run this twice due env reload
!pip install git+https://github.com/k90262/genslm

  Cloning https://github.com/k90262/genslm to c:\users\ycho\appdata\local\temp\pip-req-build-6h9opqat
  Resolved https://github.com/k90262/genslm to commit aed33b2e1492987290559e2f2f7962e0d50aff68
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/maxzvyagin/transformers to c:\users\ycho\appdata\local\temp\pip-install-whh_vxza\transformers_537d03a98f184a8cbbd7db6c89de13ab
  Resolved https://github.com/maxzvyagin/transformers to commit ffd5aba0ad41a1ebd1897a77f6a3782fc2d75e1f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/k90262/genslm 'C:\Users\ycho\AppData\Local\Temp\pip-req-build-6h9opqat'
  Running command git clone --filter=blob:none --quiet https://github.com/maxzvyagin/transformers 'C:\Users\ycho\AppData\Local\Temp\pip-install-whh_vxza\transformers_537d03a98f184a8cbbd7db6c89de13ab'


In [19]:
# Note 1: (not sure) re-install `pip install numpy==1.26.3` once facing any numpy array issues
# Note 2: we can run below command cell one by one via iPython (not must need on JupterServer)
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import svm
#from google.colab import drive
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from genslm import GenSLM, SequenceDataset

# Aquiring Model and Data

Visit: https://drive.google.com/drive/folders/1oYgda4Px-tugapgE2uumiUIf2p3PqIQI?usp=drive_link

- Right click `UmichSciFM-2024` Folder and click Organize -> Add Shortcut -> All Locations -> My Drive

Executing the cell below mounts your Google Drive to this notebook giving you access to the model checkpoint and data for this notebook

In [20]:
# Mount and see file structure
#drive.mount("/content/drive")

!dir D:\Projects\UMichSciFM-2024

 Volume in drive D is Data
 Volume Serial Number is 2A5A-AAA0

 Directory of D:\Projects\UMichSciFM-2024

08/28/2024  10:04 PM    <DIR>          .
08/28/2024  10:04 PM    <DIR>          ..
08/28/2024  10:04 PM    <DIR>          data
08/28/2024  10:04 PM    <DIR>          model
               0 File(s)              0 bytes
               4 Dir(s)  246,900,490,240 bytes free


In [21]:
# Load and view the dataset, split into train/test
data = pd.read_csv("../../UMichSciFM-2024/data/meltingpoint.csv", index_col=0)
data

,Sequence,MeltingPoint
0,atgattatttccgcagccagcgattatcgcgccgcagcacaacgca...,82.112491
1,atggctaagctgaccaagcgcatgcgcgtgatccgtgacaaagttg...,80.338892
2,atgtttaaaaataaaatgatgatttgtctttatatgtttctattat...,76.102904
3,atgggtcgactggaaggaaaggtagcgatcgtcacgggcggtgcgc...,86.743695
4,atgcgtctaaaccccggccaacaacaagctgtcgaattcgttaccg...,81.709235
...,...,...
9411,gtggatatgagtaatacaagtgcagcaccacgtgacacgtgggggt...,78.878742
9412,ttggttgagcgccacgacatcgcaaccggtgccaccgggcgtaacc...,82.666703
9413,atgttccgttcgcttcttcgcctgtctgcagcgttgctggccttga...,85.151774
9414,gtgaaattactagatttattgtcaaaaggaattgtaataggtgatg...,75.071559


In [22]:
# Split dataset for use later

# Returns two independent dataframes that we will use for
# melting point modelling
train, test = train_test_split(data, train_size=1000, test_size=200)

# Begin Modelling

Below is an example of generating embeddings with GenSLM-25M, we will follow this generat workflow to generate embeddings for our dataset, and use a downstream model to predict the melting point of an input sequence

In [23]:
# PREREQUISITES for using nvidia gpu: Setup cuda (my case as an example: GPU nvidia 1650 + win 11):
# ref. which said we still be able to run pytorch with cuda on this GPU 1650, even if this gpu only support on cuda 7.5: 
#   https://discuss.pytorch.org/t/is-there-a-table-which-shows-the-supported-cuda-version-for-every-pytorch-version/105846
# step 1. install cuda sdk 12
#   https://developer.nvidia.com/cuda-downloads?target_os=Windows&target_arch=x86_64&target_version=11&target_type=exe_local
# step 2. install pytorch which support cuda sdk 12.4 `conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia`
#   https://discuss.pytorch.org/t/is-there-a-table-which-shows-the-supported-cuda-version-for-every-pytorch-version/105846/8
# step 3. test. Re-start iPyhton, then test `import torch` and `torch.cuda.is_available()`.
#  it should return True
# done.
import torch
torch.cuda.is_available()

True

In [24]:
# hotfix issue run on windows (See: https://github.com/ultralytics/yolov5/issues/10240)
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Load model
model = GenSLM("genslm_25M_patric", model_cache_dir="../../UMichSciFM-2024/model")
model.eval()

# Select GPU device if it is available, else use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Input data is a list of gene sequences
sequences = [
    "ATGAAAGTAACCGTTGTTGGAGCAGGTGCAGTTGGTGCAAGTTGCGCAGAATATATTGCA",
    "ATTAAAGATTTCGCATCTGAAGTTGTTTTGTTAGACATTAAAGAAGGTTATGCCGAAGGT",
]

example_dataset = SequenceDataset(sequences, model.seq_length, model.tokenizer)
example_dataloader = DataLoader(example_dataset, batch_size =2)

# Compute averaged-embeddings for each input sequence
embeddings = []
with torch.no_grad():
    for batch in example_dataloader:
        outputs = model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device),
            output_hidden_states=True,
        )
        # outputs.hidden_states shape: (layers, batch_size, sequence_length, hidden_size)
        # Use the embeddings of the last layer
        emb = outputs.hidden_states[-1].detach().cpu().numpy()
        # Compute average over sequence length
        emb = np.mean(emb, axis=1)
        embeddings.append(emb)

# Concatenate embeddings into an array of shape (num_sequences, hidden_size)
embeddings = np.concatenate(embeddings)
embeddings.shape

D:\Projects\genslm\genslm\inference.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ptl_checkpoint = torch.load(weight_path, map_location="cpu")
Tokenizing...: 100%|███

(2, 512)

In [25]:
# Get embeddings for training dataset
train_dataset = SequenceDataset(train.Sequence.values, model.seq_length, model.tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8)

# Compute averaged-embeddings for each input sequence
train_embeddings = []
with torch.no_grad():
    for batch in tqdm(train_dataloader, desc="Embedding"):
        outputs = model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device),
            output_hidden_states=True,
        )
        # outputs.hidden_states shape: (layers, batch_size, sequence_length, hidden_size)
        # Use the embeddings of the last layer
        emb = outputs.hidden_states[-1].detach().cpu().numpy()
        # Compute average over sequence length
        emb = np.mean(emb, axis=1)
        train_embeddings.append(emb)

# Concatenate embeddings into an array of shape (num_sequences, hidden_size)
train_embeddings = np.concatenate(train_embeddings)
train_embeddings.shape

Embedding: 100%|██████████| 125/125 [03:41<00:00,  1.77s/it]


(1000, 512)

In [26]:
# Train SVM on embeddings for melting point
mp_regr = svm.SVR()
mp_regr.fit(train_embeddings, train.MeltingPoint.values)


SVR()

# Evaluation

In [27]:
# Get embeddings for evaluation dataset
test_dataset = SequenceDataset(test.Sequence.values, model.seq_length, model.tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Compute averaged-embeddings for each input sequence
test_embeddings = []
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Embedding"):
        outputs = model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device),
            output_hidden_states=True,
        )
        # outputs.hidden_states shape: (layers, batch_size, sequence_length, hidden_size)
        # Use the embeddings of the last layer
        emb = outputs.hidden_states[-1].detach().cpu().numpy()
        # Compute average over sequence length
        emb = np.mean(emb, axis=1)
        test_embeddings.append(emb)

# Concatenate embeddings into an array of shape (num_sequences, hidden_size)
test_embeddings = np.concatenate(test_embeddings)
test_embeddings.shape

Embedding: 100%|██████████| 25/25 [00:44<00:00,  1.77s/it]


(200, 512)

In [28]:
# Evaluate the performance of the regressor on a held out test set

r2 = mp_regr.score(test_embeddings, test.MeltingPoint.values)

print(f"Regressor R^2 {r2} for test set")

# Test a few examples and see predictions
example_predictions = mp_regr.predict(test_embeddings[:10])

for (idx, row), pred_val in zip(test.iterrows(), example_predictions):
  print(f"Empirical melting point: {row['MeltingPoint']:.3f}\t\tPredicted melting point: {pred_val:.3f}")

Regressor R^2 0.9190283094220035 for test set
Empirical melting point: 87.883		Predicted melting point: 87.494
Empirical melting point: 82.044		Predicted melting point: 84.214
Empirical melting point: 88.899		Predicted melting point: 83.655
Empirical melting point: 76.998		Predicted melting point: 78.426
Empirical melting point: 81.560		Predicted melting point: 81.449
Empirical melting point: 84.423		Predicted melting point: 85.205
Empirical melting point: 78.230		Predicted melting point: 79.274
Empirical melting point: 82.807		Predicted melting point: 81.839
Empirical melting point: 74.552		Predicted melting point: 74.406
Empirical melting point: 87.665		Predicted melting point: 86.023
